# Hypothesis Testing

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this task:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

In [14]:
# dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
path = r"C:\Users\sreej\Desktop\Jupyter_Notebook\Michigan_Data_Science\Course_1\\"

In [16]:
def get_list_of_university_towns():
    file = open(path+"university_towns.txt")
    towns = []
    for line in file:
        line = line[:-1]    
        if line[-6:] == "[edit]":
            key = line[:-6]
        elif line.find("(")!=-1:
            line = line[:line.find("(")-1]
            towns.append([key, line])
        else:
            towns.append([key, line])
            
    df = pd.DataFrame(towns, columns = ["State", "RegionName"])
    return df
get_list_of_university_towns().head(5)

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


In [18]:
rec = pd.read_excel(path+"gdplev.xls", skiprows = [0,1,2,3,4,5], header = None, usecols = "E,G", names = ["Quarter", "GDP"])
idx_start = rec[rec.Quarter == "2000q1"].index[0]
idx_end = rec.index[-1]
rec = rec.loc[range(idx_start, idx_end+1)]
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    count = rec.Quarter.count()
    rec_warn = 0
    for i in range(1, count + 1):
        if rec.GDP.iloc[i] < rec.GDP.iloc[i-1]:
            if rec_warn == 1: 
                break
            else: rec_warn = 1
        else:
            rec_warn = 0
    
    return rec.Quarter.iloc[i-1]

get_recession_start()


'2008q3'

In [19]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    rec_start = get_recession_start()
    rec_on_idx = rec[rec.Quarter==rec_start].index[0]
    idx_end = rec.index[-1]
    rec_on = rec.loc[range(rec_on_idx, idx_end+1)]
  
    count = rec_on.Quarter.count()
    rec_cool = 0
    for i in range(1, count + 1):
        if rec_on.GDP.iloc[i] > rec_on.GDP.iloc[i-1]:
            if rec_cool == 1: break
            else: rec_cool = 1
        else:
            rec_cool = 0
    
    
    return rec_on.Quarter.iloc[i]
get_recession_end()

'2009q4'

In [10]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    rec_start = get_recession_start()
    rec_end = get_recession_end()
    idx_start =  rec[rec.Quarter==rec_start].index[0]
    idx_end = rec[rec.Quarter==rec_end].index[0]
    rec_period = rec.loc[range(idx_start, idx_end+1)]
    bottom = rec_period.GDP.min()
    b_idx = rec_period[rec_period.GDP==bottom].index[0]  
    return rec_period.Quarter.loc[b_idx]


In [21]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].    
    '''   
    dfh = pd.read_csv(path+"City_Zhvi_AllHomes.csv")
    dfh["State"] = [states[st] for st in dfh["State"]]
    col = list(dfh.columns)
    #print(col)
    start = col.index("2000-01")
    col = col[start:]
    q = ["q1", "q2", "q3", "q4"]
    c = 0
    qc = 0
    year = "0000"
    df_new = pd.DataFrame()
    for i in range(len(col)):
        if ((i+1)%3 == 0): 
            #print(col[i][:4])
            df_new[str(2000+c) +q[qc]] = dfh[[col[i], col[i-1], col[i-2]]].mean(axis = 1)
            qc+= 1
            if col[i][:7]=="2016-06": 
                year = "2016"
            if qc==4:
                qc = 0
                c+=1
        elif (col[i][:4]=="2016")&(year == "2016"):
            df_new[str(2000+c) +q[qc]] = dfh[[col[i], col[i+1]]].mean(axis = 1)
            break
            
            
    df_new = df_new.set_index([dfh["State"],dfh["RegionName"]]) 
    return df_new

convert_housing_data_to_quarters().head(5)

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0


In [27]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    df_new = convert_housing_data_to_quarters()
    start = get_recession_start()
    end = get_recession_end()
    idx0 = df_new.columns.get_loc(start)
    idx1 = df_new.columns.get_loc(end)
    dfhh = df_new.iloc[:,idx0:idx1+1]
    dfhh["Ratio"] = dfhh[start]/dfhh[end]
    dfhh = dfhh["Ratio"].dropna()
    dfut = get_list_of_university_towns()
    dfhh2 = dfhh.reset_index()

    dfuniv = pd.merge(dfhh2, dfut, how = "inner", left_on = ["State", "RegionName"], right_on = ["State", "RegionName"])
    dfnonuniv = pd.merge(dfhh2, dfut, how = "left", left_on = ["State", "RegionName"], right_on = ["State", "RegionName"])
    umean = dfuniv.Ratio.mean()
    numean = dfnonuniv.Ratio.mean()
    if umean<numean:
        q = "university town"
    else:
        q = "non-university town"
    s, p = ttest_ind(dfuniv["Ratio"], dfnonuniv["Ratio"]) 
    if p<0.01:
        answer = (True, p, q )
    else:
        answer = (False, p, q )
    return answer



In [28]:
run_ttest()

C:\Users\sreej\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(True, 0.0027567593349356262, 'university town')

Null hypothesis is rejected. 